In [ ]:
from __future__ import division
import numpy as np
import matplotlib.pyplot as pl


# Define the kernel
def kernel(a, b):
    """ GP squared exponential kernel """
    kernelParameter = 0.1
    sqdist = np.sum(a**2,1).reshape(-1,1) + np.sum(b**2,1) - 2*np.dot(a, b.T)
    return np.exp(-.5 * (1/kernelParameter) * sqdist)


X = np.random.uniform(-5, 5, size=(5,1))
# print(X)
print(np.sum(X**2,1).reshape(-1,1)+np.sum(X**2,1))
print(np.sum(X**2,1))

# K = kernel(X, X)
# print(K)

In [ ]:
import torch
import math
import numpy as np

def kernel_torch(a, b):
    """ GP squared exponential kernel """
    kernelParameter = 0.1
    sqdist = torch.cdist(a,b)**2
    return torch.exp(-.5 * (1/kernelParameter) * sqdist)

N = 10         # number of training points.
s = 0.00005    # noise variance.

X = torch.FloatTensor(N, 1).uniform_(-5, 5)
y = torch.sin(X * (2 * math.pi)) + torch.randn(N)

K = kernel_torch(X, X)

L = torch.linalg.cholesky(K + s*np.eye(N))
alpha = torch.inverse(L.T) @ (torch.inverse(L) @ y.double())

f_star = 


In [10]:
from __future__ import division
import numpy as np
import matplotlib.pyplot as pl

""" This is code for simple GP regression. It assumes a zero mean GP Prior """


# This is the true unknown function we are trying to approximate
f = lambda x: np.sin(0.9*x).flatten()
#f = lambda x: (0.25*(x**2)).flatten()


# Define the kernel
def kernel(a, b):
    """ GP squared exponential kernel """
    kernelParameter = 0.1
    sqdist = np.sum(a**2,1).reshape(-1,1) + np.sum(b**2,1) - 2*np.dot(a, b.T)
    return np.exp(-.5 * (1/kernelParameter) * sqdist)

N = 10         # number of training points.
n = 50         # number of test points.
s = 0.00005    # noise variance.

# Sample some input points and noisy versions of the function evaluated at
# these points. 
X = np.random.uniform(-5, 5, size=(N,1))
y = f(X) + s*np.random.randn(N)

K = kernel(X, X)
L = np.linalg.cholesky(K + s*np.eye(N))

# points we're going to make predictions at.
Xtest = np.linspace(-5, 5, n).reshape(-1,1)

# compute the mean at our test points.
K_star = kernel(X, Xtest)
K_starstar = kernel(Xtest, Xtest)

# implementation based on algo 15.1
alpha = np.linalg.solve(L.T, np.linalg.solve(L, y)) 
f_star = np.dot(K_star.T, alpha)

# more efficient implementation?
Lk = np.linalg.solve(L, K_star)
mu = np.dot(Lk.T, np.linalg.solve(L, y))

# compute the variance at our test points.
s2 = np.diag(K_starstar) - np.sum(Lk**2, axis=0)
s = np.sqrt(s2)

print((Lk**2).shape)
print(np.sum(Lk**2, axis=0))

var = K_starstar - np.dot(Lk.T,Lk)

# print(f_star[:10])
# print(mu[:10])
print(s.shape)
print(var.shape)



(10, 50)
[2.45206211e-03 4.29396716e-02 2.94619400e-01 7.90202733e-01
 9.96838443e-01 9.98906071e-01 9.78325265e-01 9.99923784e-01
 9.96498488e-01 8.97312070e-01 4.14885406e-01 7.75375059e-02
 5.83799840e-03 1.81807952e-04 2.38877794e-06 1.34058097e-08
 3.23719907e-11 3.11618315e-12 1.61065874e-09 3.65598953e-07
 3.60083901e-05 1.53475326e-03 2.81352184e-02 2.19021139e-01
 7.10902734e-01 9.99902911e-01 9.37489849e-01 9.98422267e-01
 9.83051560e-01 9.99703404e-01 9.88308549e-01 7.13228142e-01
 2.57540649e-01 2.51860749e-01 7.17974504e-01 9.95258224e-01
 6.00648254e-01 1.57591075e-01 1.79754297e-02 8.92435361e-04
 1.02488115e-04 2.86854795e-03 4.29551158e-02 2.79662087e-01
 7.91572991e-01 9.74062910e-01 5.21100766e-01 1.21197896e-01
 1.22548251e-02 5.38713451e-04]
(50,)
(50, 50)


In [ ]:

# PLOTS:
pl.figure(1)
pl.clf()
pl.plot(X, y, 'r+', ms=20)
pl.plot(Xtest, f(Xtest), 'b-')
pl.gca().fill_between(Xtest.flat, mu-3*s, mu+3*s, color="#dddddd")
pl.plot(Xtest, mu, 'r--', lw=2)
pl.savefig('predictive.png', bbox_inches='tight')
pl.title('Mean predictions plus 3 st.deviations')
pl.axis([-5, 5, -3, 3])

# draw samples from the prior at our test points.
L = np.linalg.cholesky(K_starstar + 1e-6*np.eye(n))
f_prior = np.dot(L, np.random.normal(size=(n,10)))
pl.figure(2)
pl.clf()
pl.plot(Xtest, f_prior)
pl.title('Ten samples from the GP prior')
pl.axis([-5, 5, -3, 3])
pl.savefig('prior.png', bbox_inches='tight')

# draw samples from the posterior at our test points.
L = np.linalg.cholesky(K_starstar + 1e-6*np.eye(n) - np.dot(Lk.T, Lk))
f_post = mu.reshape(-1,1) + np.dot(L, np.random.normal(size=(n,10)))
pl.figure(3)
pl.clf()
pl.plot(Xtest, f_post)
pl.title('Ten samples from the GP posterior')
pl.axis([-5, 5, -3, 3])
pl.savefig('post.png', bbox_inches='tight')

pl.show()